In [ ]:
from pathlib import Path

import leafmap
import xarray as xr

DATA_DIR = Path("./data")

#### Download data first

We're going to work with some gridded bathymetry data!

[direct download](https://dap.ceda.ac.uk/bodc/gebco/global/gebco_2025/ice_surface_elevation/netcdf/gebco_2025.zip?download=1) |
[data info](https://www.gebco.net/data-products/gridded-bathymetry-data)

Run the cell below to download and extract this data to the `examples/data` directory!

⚠️ WARNING: this file extracts to 8gb. It will probably take **at least** 30 minutes to download.

In [ ]:
from urllib.parse import unquote, urlparse

import requests
from tqdm import tqdm


def download_file(url: str, *, dest_dir: Path = Path()) -> Path:
    """Downloads a file from a URL with a progress bar."""
    if not dest_dir.is_dir():
        raise NotADirectoryError(f"{dest_dir} must be an existing directory.")

    output_fn = unquote(urlparse(url).path).split("/")[-1]
    output_path = dest_dir / output_fn

    if output_path.is_file():
        print(f"Already downloaded {output_path}.")
        return output_path

    response = requests.get(url, stream=True, timeout=30)

    total_size = int(response.headers.get("content-length", 0))
    chunk_size = 4096

    with (
        output_path.open("wb") as f,
        tqdm(total=total_size, unit="B", unit_scale=True, desc=output_fn) as progress,
    ):
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                f.write(chunk)
                progress.update(len(chunk))
    return output_path


path = download_file(
    "https://dap.ceda.ac.uk/bodc/gebco/global/gebco_2025/ice_surface_elevation/netcdf/gebco_2025.zip?download=1",
    dest_dir=DATA_DIR,
)

In [ ]:
from shutil import unpack_archive

# TODO: Skip if already unpacked.
# TODO: Save extracted filenames.
#       Probably need to write a good amount of code with the zipfiles module?
#       ¯\_(ツ)_/¯
unpack_archive(path, extract_dir=DATA_DIR)

In [ ]:
fp = DATA_DIR / "GEBCO_2025.nc"
ds = xr.open_dataset(fp, chunks="auto")
ds

In [ ]:
elev = ds["elevation"]
elev = elev.rio.write_crs("EPSG:4326")
elev

In [ ]:
## Takes a long time, will probably use all your memory.
#
# elev.plot()  # noqa: ERA001

In [ ]:
from jupyter_xarray_tiler.titiler import add_data_array

url = await add_data_array(elev, colormap_name="terrain", rescale=(-11000, 9000))

In [ ]:
url

In [ ]:
m = leafmap.Map()
m

In [ ]:
m.add_tile_layer(url=url, name="World Bathymetry", attribution="")

## Performance

It's really slow! Especially zoomed out. It seems to use about 8GB of memory.